# Master TIDE - Conférences Python 2020

Francis Wolinski

&copy; 2020 Yotta Conseil

# 3. Pandas : manipulations et modifications des données

In [ ]:
# import des modules usuels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# commande magique pour l'affichage des graphiques
%matplotlib inline

# options d'affichage
pd.set_option("display.max_rows", 16)
plt.style.use('seaborn-darkgrid')

In [ ]:
# chargement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                  index_col="Code INSEE")
geo.sort_index(inplace=True)
geo

## 3.1 Tri
La méthode `sort_values()` permet de trier un *DataFrame* selon les valeurs d'une ou plusieurs colonnes (ordre lexicographique) et la méthode `sort_index()` selon les valeurs de l'index.

Pour trier selon l'ordre inverse on utilise l'option `ascending=False`.

Ces méthodes retournent des copies du `DataFrame` initial, sauf si l'option `inplace=True` est utilisée. Dans ce cas l'objet est effectivement modifié.

In [ ]:
# tri selon l'altitude
geo.sort_values("Altitude Moyenne").head()

In [ ]:
# tri selon l'altitude inverse
geo.sort_values("Altitude Moyenne", ascending=False).head()

In [ ]:
# tri selon l'altitude puis suivant le nom de la commune A->Z
geo.sort_values(["Altitude Moyenne", "Commune"]).head()

In [ ]:
# tri selon l'altitude puis suivant le nom de la commune Z->A
geo.sort_values(["Altitude Moyenne", "Commune"], ascending=[True, False]).head()

In [ ]:
# tri selon l'index
geo.sort_index(inplace=True)
geo.head()

## 3.2 Modification de colonnes et conversions

Toutes les opérations de sélection permettent d'effectuer des modifications avec l'opérateur `=`.

Par exemple, il est possible de modifier toutes les valeurs d'une colonne.

In [ ]:
# la superficie des communes est en hectares, on la passe en km2
geo["Superficie"] = geo["Superficie"] / 100.0  # ou bien geo["Superficie"] =/ 100.0
geo.head()

N. B. : La modification d'un objet issu d'un *DataFrame* (colonne par exemple) est répercutée sur l'objet initial.

On peut également appliquer une méthode de conversion de type sur une colonne.

In [ ]:
# exemple de conversion
geo["Altitude Moyenne"] = geo["Altitude Moyenne"].astype(int)
geo.head()

Transformation d'une variable catégorielle sous forme de chaines de caractères et categorie.

A noter, la catégorie peut être ordonnée.

In [ ]:
# info
geo.info()

In [ ]:
# conversion de la colonne Statut en catégorie

statuts = ["Commune simple", "Chef-lieu canton", "Sous-préfecture",
            "Préfecture", "Préfecture de région", "Capitale d'état"]

from pandas.api.types import CategoricalDtype
cat_statut = CategoricalDtype(categories=statuts, ordered=True)
geo["Statut"] = geo["Statut"].astype(cat_statut)

geo.info()

In [ ]:
geo.dtypes

## 3.3 Ajout de colonnes

In [ ]:
# on ajoute la colonne "Densité"
geo["Densité"] = 1000 * geo["Population"] / geo["Superficie"]
geo.head()

#### Ajout de colonnes en utilisant la méthode *apply()* appliquée à une colonne

La latitude et la longitude sont données sous la forme d'une chaîne de caractères dans la colonne *geo_point_2d*. La latitude et la longitude sont séparées par une virdule et un espace (", ")

In [ ]:
# la colonne "geo_point_2d" est constituée de chaînes de caractères
geo["geo_point_2d"]

In [ ]:
# par exemple
geo.loc["01001", "geo_point_2d"]

Pour extraire les 2 grandeurs, on va utiliser la méthode **split()** qui retourne la liste des sous-chaînes séparées par une chaîne donnée.

On va l'utiliser avec la chaîne de séparation ", ".

N.B. : Si la chaîne était variable (par ex., un ou plusieurs espaces) on pourrait utiliser une expression réguière (module *re* de Python).

In [ ]:
# application de la méthode split()
x = geo.loc["01001", "geo_point_2d"].split(', ')
x

Il va falloir extraire chacune des grandeurs et la transformer en nombre flottant.

In [ ]:
# extraction de la latitude, indice 0 dans la liste retournée par *split()*
x[0]  # on obtient une chaîne de caractères

In [ ]:
float(x[0])  # on obtient enfin un nombre flottant représentant la latitude avec le convertisseur float()

In [ ]:
'123,456,789'.split(',')

Il va falloir appliquer cette technique à chaque ligne de la colonne et aux deux grandeurs. Dans un langage classique, on appliquerait une boucle.

En pandas, la méthode *apply()* permet d'appliquer une fonction (ou une lambda) à chaque élement d'une *Series* ou d'un *DataFrame* et de retourner un objet avec le résultat de la fonction appliquée à chaque élément.

In [ ]:
# on calcule  la latitude et la longitude et on ajoute les colones
geo["Latitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[0]))
geo["Longitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[1]))
geo.head()

In [ ]:
# on vérifie le type des colonne ajoutées
geo.info()

Une autre manière de faire est d'utiliser la méthode `extract` capable d'extraire un motif sous forme d'expression régulière.

In [ ]:
# méthode extract
geo["geo_point_2d"].str.extract("(.*), (.*)", expand=True)

In [ ]:
# méthode extract
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)", expand=True).astype(float)
geo.head()

<div class="alert alert-success">
<b>Exercice 1</b>
<ul>
    <li>Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville</li>
    <li>Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.</li>
    <li>Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).</li>
</ul>
</div>

In [ ]:
# fonction recherche de ville
def ville(lat, long):
    pass

In [ ]:
# on applique la fonction à une coordonnée tirée au hasard
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

In [ ]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    pass

In [ ]:
# à partir de coordonnées GPS précises
ville(dms2dec(50, 17, 43), dms2dec(4, 6, 7))

<div class="alert alert-success">
<b>Exercice 2</b>
<br />
La colonne "geo_shape" est formée de chaines de catactères en JSON (JavaScript Object Notation). Le format JSON est très utilisé comme le format XML.

<ul>
    <li>Utiliser la librairie Python `json` pour parser la colonne `"geo_shape"` : `json.loads()`</li>
    <li>Donner le décompte des valeurs accédées avec la clé "type".</li>
    <li>Donner le décompte des longueurs des listes accédées avec la clé "coordinates".</li>
    <li>Quelle commune est la plus complexe géométriquement ?</li>
</ul>
</div>

Les formes géométriques des communes sont des polygones ou composées de plusieurs polygones.

## 3.4 Discrétisation de valeurs continues

La fonction `cut()` permet de discrétiser des variables continues. Elle renvoie un objet de type `Series` numérique qui partage le même index que l'objet Series initial.

A noter, on utilise la valeur `numpy.inf` qui désigne $+\infty$.

In [ ]:
# une ville comporte plus de 2000 habitants
var = pd.cut(geo["Population"], [0.0, 2.0, np.inf], labels=["Village", "Ville"])
var

In [ ]:
# décompte des villages et des villes de France
var.value_counts()

La fonction `qcut()` cherche à discrétiser les valeurs en quantiles identiques.

In [ ]:
# On sépare en 2 catégories identiques
var = pd.qcut(geo["Population"], 2, labels=["MonVillage", "MaVille"])
var

In [ ]:
# décompte des villages et des villes de France
var.value_counts()

In [ ]:
# seuil de la population pour MonVillage
geo.loc[var == "MonVillage", "Population"].max()

In [ ]:
# on vérifie qu'il s'agit bien de la médiane
geo["Population"].median()

In [ ]:
pd.qcut(geo["Population"], [0.0, 0.1, 0.9, 1.0])